In [1]:
! pip install tabula-py camelot-py opencv-python pandas

In [2]:
! pip install pytesseract pdf2image PyPDF2

In [ ]:
! pip install langchain_openai

In [ ]:
import os
from langchain_openai import ChatOpenAI  # Notez le changement ici

# Définition de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-0xUZ6aBpi14QWLtzQC2nF0B2gQTojxu 9HhFfHS_cUMhXQMX6_U0pycw_XiZUUtZ4V6Gc5xEwhMZOsYA6xKN4HruNnPRcA"


In [5]:
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from langchain.chat_models import ChatOpenAI
import os

class ReadFileTool(BaseTool):
    name: str = "read_file"
    description: str = "Useful for reading the content of a file."

    def _run(self, file_path: str) -> str:
        """Use the tool."""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except FileNotFoundError:
            return f"Error: File '{file_path}' not found."
        except Exception as e:
            return f"Error: {e}"

# Définir le chemin EXACT du fichier texte extrait
base_dir = r"C:\Users\kosmo\pycode\MCP_Rag_virg"
output_dir = os.path.join(base_dir, "textes_extraits")
extracted_file = os.path.join(output_dir, "2025.5.sante_pages_117-131.txt")

# Définir un agent pour l'analyse
agent_extraction = Agent(
    role="Analyste de Documents",
    goal="Analyser le contenu du fichier texte extrait et présenter les résultats dans le format demandé",
    backstory="""Vous êtes un expert en analyse de texte juridique et administratif.
    Votre spécialité est d'identifier les noms de personnes et leurs rôles
    dans des documents officiels, particulièrement les délégations de pouvoir.""",
    verbose=True,
    tools=[ReadFileTool()],
    llm=ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0
    )
)

# Définir la tâche d'analyse
tache_analyse = Task(
    description=f"""
    En utilisant l'outil read_file, lis le fichier suivant : {extracted_file}

    Analyse le texte et extrais les informations demandées, en suivant les étapes détaillées ci-dessous.

    Chain of Thought pour l'analyse

Instructions générales :
Cette méthode d'analyse doit être appliquée sur l'ensemble du document administratif en procédant article par article. Pour chaque article du document :
1. Isoler le texte de l'article concerné
2. Appliquer la méthode d'analyse décrite dans les exemples ci-dessous
3. Rechercher systématiquement les termes spécifiques et les noms associés
4. Si les termes recherchés n'existent pas dans l'article, indiquer "Non mentionné" pour la catégorie correspondante
5. Compiler les résultats en respectant le format suivant pour chaque article :

Article [Numéro]
[En cas d'absence ou d'empêchement de] : [Nom de la personne ou "Non mentionné"]
[Délégation est donnée à] : [Nom de la personne ou "Non mentionné"]

Exemple 1:

1. Lecture du document :
   Je commence par lire attentivement l'extrait suivant pour en comprendre le contexte et la structure.

   "Délégation est donnée à M. Thomas DUPONT, responsable de l'Unité budget et contrôle interne au sein de la

   Direction des achats et des finances, à l'effet de signer, au nom de la directrice générale de Santé publique France,
   dans la limite de ses attributions et fonctions :

   - l'ensemble des bons de commande d'un montant hors taxe inférieur à 35 000 € ;
   - en cas d'absence ou d'empêchement de la directrice des achats et des finances,
   Mme Sophie MARTIN (épouse DURAND), l'ensemble des bons de commande ;
   - les certifications de service fait sans limitation de montant."

2. Recherche du terme "délégation est donnée" :
   Je cherche si cette expression apparaît dans le texte et j'examine ce qui suit.

   Trouvé : "Délégation est donnée à M. Thomas DUPONT"

   J'identifie donc le nom qui suit cette expression : M. Thomas DUPONT

   Si cette expression n'était pas présente, j'indiquerais "Non mentionné".

   Si cette expression est présente :

   j'identifie la fonction devant le nom de M. Thomas DUPONT : responsable de l'Unité budget et contrôle interne au sein de la Direction des achats et des finances

   la fonction est :  responsable de l'Unité budget et contrôle interne au sein de la Direction des achats et des finances

3. Recherche du terme "en cas d'absence ou d'empêchement" :
   Je cherche si cette expression apparaît dans le texte et j'examine le contexte.

   Trouvé : "en cas d'absence ou d'empêchement de la directrice des achats et des finances, Mme Sophie MARTIN (épouse DURAND)"

   Dans ce contexte, je comprends que Mme Sophie MARTIN (épouse DURAND) est mentionnée comme la directrice des achats et des finances.

   Si cette expression n'était pas présente, j'indiquerais "Non mentionné".

   Si cette expression est présente j'identife également la fonction devant le nom de Mme Sophie MARTIN (épouse DURAND) : directrice des achats et des finances

   la fonction est : directrice des achats et des finances



4. Format de sortie :
   Je structure les informations collectées selon le format demandé.

   [En cas d'absence ou d'empêchement de] : Mme Sophie MARTIN (épouse DURAND) , fonction : directrice des achats et des finances
   [Délégation est donnée à] : M. Thomas DUPONT , fonction : responsable de l'Unité budget et contrôle interne au sein de la Direction des achats et des finances

Exemple 2:

1. Lecture du document :
   Je commence par lire attentivement l'extrait suivant pour en comprendre le contexte et la structure.

   "En cas d'absence ou d'empêchement de Mme Sophia DUBOIS, directrice de l'aide et diffusion aux publics, délégation est donnée à Mme Camille LAURENT, adjointe, dans la limite de ses attributions et fonctions :

   - les engagements financiers relatifs à l'activité de la

   Direction de l'aide et diffusion aux publics d'un montant hors taxe inférieur à 25 000 € et les engagements contractuels afférents ;
   - les lettres de mission envoyées à des collaborateurs externes pour la relecture de rapports produits par Santé publique France avec la mention du montant d'indemnisation de la vacation ;
   - toute décision relative aux opérations d'inventaire dans le cadre de l'arrêté annuel des comptes."

2. Recherche du terme "En cas d'absence ou d'empêchement de" :
   Je cherche cette expression dans le texte et j'examine ce qui suit.

   Trouvé : "En cas d'absence ou d'empêchement de Mme Sophia DUBOIS"

   J'identifie donc le nom qui suit cette expression : Mme Sophia DUBOIS

   Si cette expression n'était pas présente, j'indiquerais "Non mentionné".

   Si cette expression est présente j'identife également la fonction devant le nom de Mme Sophia DUBOIS : directrice de l'aide et diffusion aux publics

   la fonction est : directrice de l'aide et diffusion aux publics

3. Recherche du terme "délégation est donnée à" :
   Je cherche cette expression dans le texte et j'examine ce qui suit.

   Trouvé : "délégation est donnée à Mme Camille LAURENT"

   J'identifie donc le nom qui suit cette expression : Mme Camille LAURENT

   Si cette expression n'était pas présente, j'indiquerais "Non mentionné".

   Si cette expression est présente j'identife également la fonction devant le nom de Mme Camille LAURENT : adjointe

   la fonction est : adjointe

4. Format de sortie :
   Je structure les informations collectées selon le format demandé.

   [En cas d'absence ou d'empêchement de] : Mme Sophia DUBOIS fonction : directrice de l'aide et diffusion aux publics
   [Délégation est donnée à] : Mme Camille LAURENT fonction : adjointe

Le résultat final pour l'ensemble du document ressemblera à ceci :

Article 22
[En cas d'absence ou d'empêchement de] : Mme Sophia DUBOIS fonction : directrice de l'aide et diffusion aux publics
[Délégation est donnée à] : Mme Camille LAURENT fonction : adjointe

Article 8
[En cas d'absence ou d'empêchement de] : Mme Sophie MARTIN (épouse DURAND) fonction : directrice des achats et des finances
[Délégation est donnée à] : M. Thomas DUPONT fonction : responsable de l'Unité budget et contrôle interne au sein de la Direction des achats et des finances

Article 15
[En cas d'absence ou d'empêchement de] : Non mentionné
[Délégation est donnée à] : Non mentionné
    """,
    agent=agent_extraction,
    expected_output="Les informations de délégation extraites du document selon le format spécifié"
)

# Configurer l'équipe avec un seul agent et une seule tâche
crew = Crew(
    agents=[agent_extraction],
    tasks=[tache_analyse],
    verbose=True
)

# Exécuter l'équipe
result = crew.kickoff()

print("\n======== RÉSULTAT DE L'ANALYSE ========")
print(result)

C:\Users\kosmo\AppData\Local\Temp\ipykernel_25304\4085261108.py:34: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(


LLM created with extracted parameters; model='gpt-4o-mini'
# Agent: Analyste de Documents
## Task: 
    En utilisant l'outil read_file, lis le fichier suivant : C:\Users\kosmo\pycode\MCP_Rag_virg\textes_extraits\2025.5.sante_pages_117-131.txt

    Analyse le texte et extrais les informations demandées, en suivant les étapes détaillées ci-dessous.

    Chain of Thought pour l'analyse

Instructions générales :
Cette méthode d'analyse doit être appliquée sur l'ensemble du document administratif en procédant article par article. Pour chaque article du document :
1. Isoler le texte de l'article concerné
2. Appliquer la méthode d'analyse décrite dans les exemples ci-dessous
3. Rechercher systématiquement les termes spécifiques et les noms associés
4. Si les termes recherchés n'existent pas dans l'article, indiquer "Non mentionné" pour la catégorie correspondante
5. Compiler les résultats en respectant le format suivant pour chaque article :

Article [Numéro]
[En cas d'absence ou d'empêchemen

In [ ]:
#FFFFF